In [1]:
var nums = 1  to 10;

nums = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [2]:
var numsRDD = sc.parallelize(nums)

numsRDD = ParallelCollectionRDD[0] at parallelize at <console>:29


ParallelCollectionRDD[0] at parallelize at <console>:29

In [3]:
numsRDD.partitions.length

6

In [4]:
numsRDD.saveAsTextFile("somedir")

[Stage 0:>                                                          (0 + 6) / 6]

In [5]:
numsRDD.glom().collect()

[[1], [2, 3], [4, 5], [6], [7, 8], [9, 10]]

In [8]:
var kvrdd = numsRDD.map((_, 1));

kvrdd = MapPartitionsRDD[3] at map at <console>:31


MapPartitionsRDD[3] at map at <console>:31

In [9]:
kvrdd.collect()

[(1,1), (2,1), (3,1), (4,1), (5,1), (6,1), (7,1), (8,1), (9,1), (10,1)]

In [10]:
import org.apache.spark.HashPartitioner

In [11]:
var prdd = kvrdd.partitionBy(new HashPartitioner(3));

prdd = ShuffledRDD[4] at partitionBy at <console>:34


ShuffledRDD[4] at partitionBy at <console>:34

In [12]:
prdd.glom().collect()

[[(3,1), (6,1), (9,1)], [(1,1), (4,1), (7,1), (10,1)], [(2,1), (5,1), (8,1)]]

In [13]:
import org.apache.spark.RangePartitioner
var words = sc.parallelize(Array("this", "is", "a", "cat", "there", "is", "a", "rat", "cat", "eats", "rat"), 2)
var wordsTups = words.map((_, 1))
var output = wordsTups.partitionBy(new RangePartitioner(2, wordsTups))
output.glom().collect()

words = ParallelCollectionRDD[6] at parallelize at <console>:29
wordsTups = MapPartitionsRDD[7] at map at <console>:30
output = ShuffledRDD[10] at partitionBy at <console>:31


[[(is,1), (a,1), (cat,1), (is,1), (a,1), (cat,1), (eats,1)], [(this,1), (there,1), (rat,1), (rat,1)]]

In [14]:
output.saveAsTextFile("myoutput_dir")

In [1]:
sc.setLogLevel("ERROR")

In [2]:
var file = sc.textFile("/data/mr/wordcount/input/")

file = /data/mr/wordcount/input/ MapPartitionsRDD[1] at textFile at <console>:27


/data/mr/wordcount/input/ MapPartitionsRDD[1] at textFile at <console>:27

In [3]:
var numBlankLines = sc.accumulator(0)

numBlankLines = 0


0

In [4]:
def toWords(line:String): Array[String] = {    
  if(line.length == 0) {numBlankLines += 1}    
  return line.split(" ");    
}


toWords: (line: String)Array[String]


In [5]:
var words = file.flatMap(toWords)
words.saveAsTextFile("words3")

[Stage 0:===================>                                       (1 + 2) / 3]

words = MapPartitionsRDD[2] at flatMap at <console>:33


MapPartitionsRDD[2] at flatMap at <console>:33

In [6]:
printf("Blank lines: %d", numBlankLines.value)    
//Blank lines: 24858

Blank lines: 24858

In [7]:
class MyComplex(var x: Int, var y: Int) extends Serializable{
def reset(): Unit = {
    x = 0
    y = 0
}
def add(p:MyComplex): MyComplex = {
    x = x + p.x
    y = y + p.y
    return this
}
}

import org.apache.spark.AccumulatorParam
class ComplexAccumulatorV1 extends AccumulatorParam[MyComplex] {

    def zero(initialVal: MyComplex): MyComplex = {
        return initialVal
    }

    def addInPlace(v1: MyComplex, v2: MyComplex): MyComplex = {
        v1.add(v2)
        return v1;
    }
}

val vecAccum = sc.accumulator(new MyComplex(0,0))(new ComplexAccumulatorV1)

var myrdd = sc.parallelize(Array(1,2,3))
def myfunc(x:Int):Int = {
    vecAccum += new MyComplex(x, x)
    return x * 3
}
var myrdd1 = myrdd.map(myfunc)
myrdd1.collect()
vecAccum.value.x
vecAccum.value.y

defined class MyComplex
defined class ComplexAccumulatorV1
vecAccum = MyComplex@26ff3e45
myrdd = ParallelCollectionRDD[4] at parallelize at <console>:42
myrdd1 = MapPartitionsRDD[5] at map at <console>:47


myfunc: (x: Int)Int


6